In [62]:
import together
from dotenv import load_dotenv
import os
import time
import pickle

### Fintune

In [63]:
load_dotenv()
together.api_key = os.getenv('TOGETHER_API_KEY')

In [64]:
resp = together.Files.check(file="cipher_data.jsonl")
print(resp)

{'is_check_passed': True, 'model_special_tokens': 'we are not yet checking end of sentence tokens for this model', 'file_present': 'File found', 'file_size': 'File size 0.001 GB', 'num_samples': 9600}


In [65]:
resp = together.Files.upload(file="cipher_data.jsonl")
print(resp)

Uploading cipher_data.jsonl: 100%|██████████| 1.09M/1.09M [00:00<00:00, 1.29MB/s]

{'filename': 'cipher_data.jsonl', 'id': 'file-6f6a2a4f-44c1-4589-b262-da772916a495', 'object': 'file', 'report_dict': {'is_check_passed': True, 'model_special_tokens': 'we are not yet checking end of sentence tokens for this model', 'file_present': 'File found', 'file_size': 'File size 0.001 GB', 'num_samples': 9600}}


In [69]:
fine_tune_resp = together.Finetune.create(
  training_file = resp['id'],
  model = 'togethercomputer/llama-2-7b-chat',
  n_epochs = 4,
  n_checkpoints = 1,
  batch_size = 32,
  # lora = True, #super important
  learning_rate = 1e-5,
  suffix = 'cs224n-milestone-finetune',
  wandb_api_key = os.getenv('WAND_DB')
)

In [70]:
fine_tune_resp

{'training_file': 'file-6f6a2a4f-44c1-4589-b262-da772916a495',
 'validation_file': '',
 'model_output_name': 'macvincentaghaoko@gmail.com/llama-2-7b-chat-cs224n-milestone-finetune-2024-03-02-00-58-50',
 'model_output_path': 's3://together-dev/finetune/65df9e44f5850c89099e8634/macvincentaghaoko@gmail.com/llama-2-7b-chat-cs224n-milestone-finetune-2024-03-02-00-58-50/ft-6f11d58a-662c-4faf-a3bd-e07226e2a19c',
 'Suffix': 'cs224n-milestone-finetune',
 'model': 'togethercomputer/llama-2-7b-chat',
 'n_epochs': 4,
 'n_checkpoints': 1,
 'batch_size': 32,
 'learning_rate': 1e-05,
 'eval_steps': 0,
 'user_id': '65df9e44f5850c89099e8634',
 'lora': False,
 'lora_r': 8,
 'lora_alpha': 8,
 'lora_dropout': 0,
 'staring_epoch': 0,
 'training_offset': 0,
 'checkspoint_path': '',
 'random_seed': '',
 'created_at': '2024-03-02T00:58:50.476Z',
 'updated_at': '2024-03-02T00:58:50.476Z',
 'status': 'pending',
 'owner_address': '0xdfcc9c9ecdd66266dfc6d8fc11d2b8eb0622e840',
 'id': 'ft-6f11d58a-662c-4faf-a3bd-e0

In [76]:

fine_tune_id = fine_tune_resp['id']
print(together.Finetune.get_job_status(fine_tune_id=fine_tune_id)) # pending, running, completed, compressing
print(together.Finetune.is_final_model_available(fine_tune_id=fine_tune_id)) # True, False

running
False


In [74]:
username = 'amacvincent'
wandb_url = f'https://wandb.ai/{username}/together?workspace=user-{username}'
print(wandb_url)

https://wandb.ai/amacvincent/together?workspace=user-amacvincent


### Inference

In [35]:
model_name = fine_tune_resp["model_output_name"]
together.Models.start(model_name)

{'success': True,
 'value': '8a35fc77f1934b85183184fdeaa66566dcbf7060a07a0b7cf5ba8deab057418b-56765a6fe2bb6a79ec81ccd91a4b56fa55600232925861afda937a13f7a6f2e7'}

In [61]:
output = together.Complete.create(
  prompt = "[Ciphered]: dah to go thru eht <unk> to take it back .\n[Deciphered]: ", 
  model = model_name,
  temperature = 0.2,
  top_k = 1,
  top_p = 1,
  repetition_penalty = 1.1,
  stop = ['</s>']
)

In [59]:
output

{'status': 'finished',
 'prompt': ['[Ciphered]: dah to go thru eht <unk> to take it back .\n[Deciphered]: '],
 'model': 'macvincentaghaoko@gmail.com/llama-2-7b-chat-cs224n-milestone-finetune-2024-03-01-23-43-49',
 'model_owner': '',
 'tags': {},
 'num_returns': 1,
 'args': {'model': 'macvincentaghaoko@gmail.com/llama-2-7b-chat-cs224n-milestone-finetune-2024-03-01-23-43-49',
  'prompt': '[Ciphered]: dah to go thru eht <unk> to take it back .\n[Deciphered]: ',
  'top_p': 1,
  'top_k': 1,
  'temperature': 0.2,
  'max_tokens': 128,
  'stop': ['</s>'],
  'repetition_penalty': 1.1,
  'logprobs': None,
  'safety_model': None},
 'subjobs': [],
 'output': {'choices': [{'text': '"dah to go through EHT to take it back."\n\nWhat does this message mean?'}],
  'request_id': '85dd684cfe8e2314-SJC',
  'usage': {'completion_tokens': 22, 'prompt_tokens': 28, 'total_tokens': 50}}}

In [52]:
with open('test_dataset.pickle', 'rb') as handle:
    test_dataset = pickle.load(handle)
test_input, test_output = test_dataset

In [54]:
print(len(test_input), len(test_output))

2400 2400
